In [1]:
import os
import time
from datetime import datetime
from IPython.display import Markdown, display
from lxml import etree
import openai
import json
import re
import tqdm

In [2]:
start_time = time.time()

In [3]:
def display_completion_content(content):
    display(Markdown(content))

In [4]:
def pre_process(file_path):
    with open(file_path, 'rb') as file:
        xml_content = file.read()
    root = etree.fromstring(xml_content)
    for bpmndi_tag in root.xpath(".//bpmndi:*", namespaces={'bpmndi': 'http://www.omg.org/spec/BPMN/20100524/DI'}):
        parent = bpmndi_tag.getparent()
        if parent is not None:
            parent.remove(bpmndi_tag)
    modified_xml = etree.tostring(root, pretty_print=True, encoding='unicode')
    return modified_xml

In [5]:
# Configure OpenAI client
client = openai.Client(api_key="Your OPENAI API Key")

In [15]:
# Base directories and filenames
Model = "PaperModel.bpmnml"
Patter_Jsonl ="All_Conflcit_Patterns.jsonl"
Prompt ="Prompt.txt"
output_base_folder = "EVAL_Results"


In [13]:
Model = pre_process(Model)
Counter = 1

In [14]:
with open(Patter_Jsonl, 'r', encoding='utf-8') as file:
    lines = file.readlines()
     
    for line_number, line in tqdm.tqdm(enumerate(lines, start=1), total=len(lines), desc="Processing"):
        # for line_number, line in enumerate(file, start=1):
        data = json.loads(line.strip())
        filename = data.get('filename', None)  # Fallback to 'Unknown' if the key is missing
        XML_Contents = data.get('XML_Contents', None)
        
        # Load the default prompt
        with open(Prompt, 'r') as file:
            default_system_prompt = file.read()
        
        default_prompt = f"{default_system_prompt}\nConflict Pattern Name: {filename}\nConflict Pattern: {XML_Contents}\nModel: {Model}"

        user_prompt = '''
        Is there a conflict detected based on the criteria described in the provided XML content?If so, explain what the conflict means for the use case, including which requirement is conflicting and why it is a conflict, and suggest a resolution.
        '''
        messages = [{"role": "system", "content": default_prompt}, {"role": "user", "content": user_prompt}]

        llm_start_time = time.time()  # LLM call start time
        
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        llm_end_time = time.time()  # LLM call end time

        completion_content = completion.choices[0].message.content
        llm_response_time = round(llm_end_time - llm_start_time)

        #writting each api chat compeltion to a File 
        output_name = str(Counter)+ ".txt"
        output_name = os.path.join(output_base_folder, output_name)

        try:
            with open(output_name, "w") as file:
                completion_content = f"File Name: {output_name}\n Pattern ID :{Counter}\nPattern Name: {filename}\n{completion_content}\nLLM Response Time: {str(llm_response_time)}"
                # Encoding to 'utf-8' and decoding with 'ignore' to remove unknown Unicode characters
                safe_content = completion_content.encode('utf-8', 'ignore').decode('utf-8')
                file.write(safe_content)
        except Exception as error:  # Using 'error' instead of 'e' as the variable name
            print(f"An error occurred while writing to the file: {error}")

        print(str(Counter)+" LLM time : " + str(llm_response_time))

        Counter+=1

Processing:  50%|█████     | 1/2 [00:13<00:13, 13.75s/it]

1 LLM time : 14


Processing: 100%|██████████| 2/2 [00:32<00:00, 16.49s/it]

2 LLM time : 19


In [9]:
end_time = time.time()
total_runtime = end_time - start_time

print(total_runtime)

27.41800093650818


In [10]:
print(round(total_runtime/60))

0
